# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

from typing import *

import reader
from bs4 import BeautifulSoup
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()
import logging

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, "html.parser")
    return s.text

In [ ]:
# | export


class Feed:
    """RSS feed class"""

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)

In [ ]:
# | export

import rich


class PyNewsReader:
    def __init__(self, dbpath=None, feeds=List[Feed]):
        if dbpath is None:
            logger.info("Database path not specified, using ./db.sqlite")
            self.dbpath = "db.sqlite"
        else:
            self.dbpath = dbpath

        self._reader = reader.make_reader(
            self.dbpath, plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"]
        )

        self._reader.enable_search()
        self._feed_names = {}

    def _print_entries(
        self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10
    ):
        """
        Pretty print entries - supports reader.Reader.get_entries arguments
        """
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = "Date: " + e.published.isoformat()[:10]
                else:
                    published_date = "Date: Unknown"
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = f"[bold]{self._get_feed_title(e.original_feed_url)}[/bold]"
                panel_body = f"Title: [bold]{e.title}[/bold]" + "\n"
                panel_body += str(published_date) + "\n\n"
                panel_body += strip_html(e.summary).strip() + "\n"

                console.print(
                    Panel(
                        panel_body,
                        title=feed_title,
                        subtitle=f"[link={e.link}]{e.link}[/link]",
                    )
                )
                console.print()
            if len(displayed_links) == limit:
                return

    def _search_to_entry(self, search_result):
        for i in self._reader.get_entries():
            if i.id == search_result.id and i.feed_url == search_result.feed_url:
                return i

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed

        Args:
            url (str): URL of pynewsreader feed

        Returns:
            str: Display title
        """
        if url in self._feed_names:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def update(self):
        """Update feeds and search"""
        self._reader.update_feeds()
        self._reader.update_search()

    def _get_entries(
        self, important: bool = None, read: Union[None, bool] = None, limit: int = 10
    ):
        """Get entries in reader.Entry format

        Args:
            read (reader.Entry.read, optional): Filter on `read` status (None, True, False). Defaults to None.
            limit (int, optional): Number of entries to return. Defaults to 10.

        Returns:
            List[reader.Entry]: List of entries
        """
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def _get_tags(self, entry: reader.Entry):
        """Get tags for a given entry"""
        # To set a tag:
        # r._reader.set_tag(test[0], "foobar")
        # To delete a tag:
        # r._reader.delete_tag(test[0], "foobar")
        return [i[0] for i in list(self._reader.get_tags(entry))]

    def add_feed(self, feed: Union[Feed, str]):
        """Add feed to pynewsreader

        Args:
            feed (Feed): pynewsreader Feed to add
        """
        if isinstance(feed, Feed):
            self._feed_names[feed.url] = feed.name
            self._reader.add_feed(feed.url, exist_ok=True)
        elif isinstance(feed, str):
            self._reader.add_feed(feed, exist_ok=True)
        else:
            raise Exception("Must be str or Feed type to add")
            

    def remove_feed(self, feed: Feed):
        """Remove feed from pynewsreader instance

        Args:
            feed (Feed): Feed to remove
        """

        self._reader.delete_feed(feed.url)

    def feeds(self):
        """List pynewsreader feeds

        Returns:
            List[str]: List of names of current pynewsreader feeds
        """
        return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

    def show(
        self,
        limit: int = 6,
        read: bool = False,
        important: bool = None,
        mark_as_read: bool = True,
    ):
        """Pretty print entries

        Args:
            limit (int, optional): Number of entries to show. Defaults to 5.
            read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
            mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
        """
        self._print_entries(
            self._get_entries(read=read, important=important, limit=limit * 2),
            limit=limit,
            mark_as_read=mark_as_read,
        )

    def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
        """Search entries and pretty print results

        Args:
            query (str): Search query
            mark_as_read (bool, optional): Mark results as read? Defaults to True.
        """
        self._print_entries(
            [self._search_to_entry(i) for i in self._reader.search_entries(query)],
            mark_as_read=mark_as_read,
            limit=limit,
        )

    def _mark_important(self, entry: reader.Entry):
        """Mark entry as important

        Args:
            entry (reader.Entry): Entry to mark as important
        """
        reader.Reader.mark_entry_as_important(entry)

    def _mark_unimportant(self, entry: reader.Entry):
        """Mark entry as important

        Args:
            entry (reader.Entry): Entry to mark as important
        """
        reader.Reader.mark_entry_as_unimportant(entry)

    def _add_tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Add tag to entry

        Args:
            entry (reader.Entry): Entry to tag
            tag_key (str): Key of tag
            tag_value (Dict, optional): Value of tag. Defaults to None.
        """
        reader.Reader.set_tag(entry, tag_key, tag_value)

    def _remove_tag(self, entry: reader.Entry, tag_key: str):
        """Remove tag from entry

        Args:
            entry (reader.Entry): Entry to tag
            tag_key (str): Key of tag
        """
        self._reader.delete_tag(entry, tag_key)

In [ ]:
r = PyNewsReader()

## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url="https://ricochet.media/en/feed", name="Richochet Media"),
    Feed(url="https://thetyee.ca/rss2.xml", name="The Tyee"),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
        name="Toronto Star | Top Stories",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
        name="Toronto Star | Investigations",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
        name="Toronto Star | Editorials",
    ),
    Feed(url="https://www.macleans.ca/feed/", name="Macleans"),
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://thenarwhal.ca/feed/",
]:
    r.add_feed(Feed(i))

## List Feeds

In [ ]:
r.feeds()

['Richochet Media',
 None,
 None,
 None,
 None,
 None,
 None,
 'Macleans',
 None,
 'The Tyee',
 'Toronto Star | Investigations',
 'Toronto Star | Editorials',
 'Toronto Star | Top Stories']

## Update Feeds

In [ ]:
r.update()

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(limit=5, read=None, mark_as_read=False)

╭───────────────────────────────────────────────────── None ──────────────────────────────────────────────────────╮
│ Title: Canada among 4 countries launching effort to hold Iran accountable for shooting down Flight PS752        │
│ Date: 2022-12-28                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰──────────── ]8;id=863702;https://www.cbc.ca/news/politics/canada-trigger-convention-iran-ps752-1.6698758?cmp=rss\https://www.cbc.ca/news/politics/canada-trigger-convention-iran-ps752-1.6698758?cmp=rss]8;;\ ────────────╯

╭───────────────────────────────────────────────────── None ──────────────────────────────────────────────────────╮
│ Title: NASA mulls SpaceX backup plan for crew of Russia's leaky Soyuz ship                                      │
│ Date: 2022-12-28                                                                                                │
│                                                                                                                 │
│ Following a coolant leak on a Soyuz capsule earlier this month, three members of the International Space        │
│ Station might need an alternative ride home.                                                                    │
│                                                                                                                 │
╰──────────── ]8;id=247281;https://www.cbc.ca/news/science/nasa-spacex-russia-soyuz-coolant-leak-1.6698775?cmp=rss\https://www.cbc.ca/news/science/nasa-spacex-russia-soyuz-coolant-leak-1.6698775?cmp=rss]8;;\ ────────────╯

╭───────────────────────────────────────────────────── None ──────────────────────────────────────────────────────╮
│ Title: Kingston Police appoint interim chief as former chief set to retire                                      │
│ Date: 2022-12-28                                                                                                │
│                                                                                                                 │
│ Kingston will start the new year with a new interim police chief, as former Chief Antje McNeely steps down from │
│ the role before easing into retirement.                                                                         │
│                                                                                                                 │
╰───────────── ]8;id=566991;https://www.cbc.ca/news/canada/ottawa/kingston-police-interim-chief-1.6698740?cmp=rss\https://www.cbc.ca/news/canada/ottawa/kingston-police-interim-chief-1.6698740?cmp=rss]8;;\ ─────────────╯

╭───────────────────────────────────────────────────── None ──────────────────────────────────────────────────────╮
│ Title: Skip the doctor, see a pharmacist: 13 conditions you can have treated at an Ontario pharmacy in 2023     │
│ Date: 2022-12-28                                                                                                │
│                                                                                                                 │
│ Ontario pharmacies will officially be able to assess and prescribe medication for 13 common ailments as of Jan. │
│ 1. People will not be required to pay out of pocket for pharmacist visits.                                      │
│                                                                                                                 │
╰─────────── ]8;id=795118;https://www.cbc.ca/news/canada/toronto/ontario-pharmacies-prescriptions-1.6698703?cmp=rss\https://www.cbc.ca/news/canada/toronto/ontario-pharmacies-prescriptions-1.6698703?cmp=rss]8;;\ ───────────╯

╭───────────────────────────────────────────────────── None ──────────────────────────────────────────────────────╮
│ Title: 1 killed, 9 injured in 14-vehicle crash north of Calgary                                                 │
│ Date: 2022-12-28                                                                                                │
│                                                                                                                 │
│ One person was killed and nine people were injured after a 14-vehicle collision north of Calgary late Tuesday   │
│ night, RCMP said in releases.                                                                                   │
│                                                                                                                 │
╰─ ]8;id=443600;https://www.cbc.ca/news/canada/calgary/fires-injuries-reported-after-massive-multi-vehicle-collision-north-of-calgary-1.6698649?cmp=rss\https://www.cbc.ca/news/canada/calgary/fires-injuries-reported-after-massive-multi-vehicle-collision-north-of-]8;;\─╯

## Search Entries

In [ ]:
r.search("fusion")

╭───────────────────────────────────────────────────── None ──────────────────────────────────────────────────────╮
│ Title: Why burning plasma could be the next milestone in nuclear fusion research                                │
│ Date: 2022-12-16                                                                                                │
│                                                                                                                 │
│ Bob McDonald's blog: A laser fusion experiment in the U.S. produced more energy out than went into driving it,  │
│ which is an important step. But practical fusion power will likely come from a different technology and is      │
│ still many years away.                                                                                          │
│                                                                                                                 │
╰─ ]8;id=642260;https://www.cbc.ca/radio/quirks/why-burning-plasma-could-be-the-next-milestone-in-nuclear-fusion-research-1.6687322?cmp=rss\https://www.cbc.ca/radio/quirks/why-burning-plasma-could-be-the-next-milestone-in-nuclear-fusion-research-1.66]8;;\─╯

╭───────────────────────────────────────────────────── None ──────────────────────────────────────────────────────╮
│ Title: Fusion energy 'breakthrough' revealed by U.S. scientists                                                 │
│ Date: 2022-12-13                                                                                                │
│                                                                                                                 │
│ U.S. Energy Secretary Jennifer Granholm announced a "major scientific breakthrough" on Tuesday in the           │
│ decades-long quest to harness fusion, the energy that powers the sun and stars.                                 │
│                                                                                                                 │
╰─────────────────── ]8;id=358437;https://www.cbc.ca/news/science/us-fusion-announcement-1.6682497?cmp=rss\https://www.cbc.ca/news/science/us-fusion-announcement-1.6682497?cmp=rss]8;;\ ────────────────────╯

╭───────────────────────────────────────────────────── None ──────────────────────────────────────────────────────╮
│ Title: What is nuclear fusion and why is it such a big deal?                                                    │
│ Date: 2022-12-13                                                                                                │
│                                                                                                                 │
│ The U.S. Department of Energy on Tuesday announced a breakthrough in nuclear fusion, a way of producing clean   │
│ energy that scientists have been working on since the 1940s. So what exactly is it, why is it so important and  │
│ what does it mean for the future?                                                                               │
│                                                                                                                 │
╰────────────────── ]8;id=145332;https://www.cbc.ca/news/science/nuclear-fusion-explainer-1.6684298?cmp=rss\https://www.cbc.ca/news/science/nuclear-fusion-explainer-1.6684298?cmp=rss]8;;\ ───────────────────╯

╭───────────────────────────────────────────────────── None ──────────────────────────────────────────────────────╮
│ Title: The cannabis industry has a plastic waste problem — but some are finding solutions                       │
│ Date: 2022-12-15                                                                                                │
│                                                                                                                 │
│ In this week's issue of our environment newsletter, we look at the plastic waste produced in the cannabis       │
│ industry and what this week's announcement about nuclear fusion means for the future of clean energy.           │
│                                                                                                                 │
╰──────────── ]8;id=596910;https://www.cbc.ca/news/science/what-on-earth-cannabis-plastic-waste-1.6687271?cmp=rss\https://www.cbc.ca/news/science/what-on-earth-cannabis-plastic-waste-1.6687271?cmp=rss]8;;\ ─────────────╯

## Tag Entry

In [ ]:
test = list(r._reader.get_entries())

In [ ]:
r._reader.set_tag(test[0], "foobar")

In [ ]:
list(r._reader.get_tags(test[0]))

[('foobar', None)]

In [ ]:
# We added a method to just return the tag key:
r._get_tags(test[0])

['foobar']

## Mark as Important

In [ ]:
r._reader.mark_entry_as_important(test[0])

## Mark as unimportant

In [ ]:
r._reader.mark_entry_as_unimportant(test[0])

## Mark as Read

In [ ]:
r._reader.mark_entry_as_read(test[0])

## Remove tag from entry

In [ ]:
r._reader.delete_tag(test[0], "foobar")

In [ ]:
list(r._get_tags(test[0]))

[]

## Automatically mark entries as read

In [ ]:
def mark_matching_entries_as_read(r, match_strings=[]):
    assert isinstance(match_strings, list)
    for i in r._reader.get_entries():
        for filter_string in match_strings:
            if filter_string in i.title:
                r._reader.mark_entry_as_read(i)

In [ ]:
%%time
mark_matching_entries_as_read(r, ["Apple"])

CPU times: user 35.3 ms, sys: 0 ns, total: 35.3 ms
Wall time: 45.7 ms


## CLI

In [ ]:
# | export
import fire


def main():
    fire.Fire(PyNewsReader)


In [ ]:
# | hide

# To Do:
# * Functions to mark entries as read before actually seeing them - accept an interable or entries and mark entries
# * User interface
# * Use pathlib to store database in ~.cache/pynewsreader

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()